In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 17259547
paper_name = 'zakrzewska_klis_2007' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
sheets = ['5h_non-essential','5h_essential','9h_non-essential','9h_essential']

In [7]:
original_data_list = []
for s in sheets:
    original_data = pd.read_excel('raw_data/SD1.xlsx', sheet_name=s)
    print('Original data dimensions: %d x %d' % (original_data.shape))
    print(original_data.head())
    original_data['orf'] = original_data['ORF NAME'].astype(str)
    original_data['orf'] = clean_orf(original_data['orf'])
    original_data['orf'] = translate_sc(original_data['orf'], to='orf')
    t = looks_like_orf(original_data['orf'])
    print(original_data.loc[~t,])
    original_data['data'] = pd.to_numeric(original_data.iloc[:,1], errors='coerce')
    original_data.set_index('orf', inplace=True)
    original_data = original_data[['data']].copy()
    original_data = original_data.groupby(original_data.index).mean()
    
    print(original_data.shape)
    original_data_list.append(original_data)

Original data dimensions: 4587 x 2
  ORF NAME  log2 ratio after a 5-h exposure to chitosan
0  YBR274W                                     9.574846
1  YJR118C                                     7.642990
2  YGR272C                                     6.910552
3  YGL084C                                     6.465861
4  YGR165W                                     6.313246
Empty DataFrame
Columns: [ORF NAME, log2 ratio after a 5-h exposure to chitosan, orf]
Index: []
(4527, 1)
Original data dimensions: 1073 x 2
  ORF NAME  log2 ratio after a 5-h exposure to chitosan
0  YCL003W                                     1.458425
1  YDL007W                                     1.382834
2  YBR029C                                     1.335235
3  YFL002C                                     1.288893
4  YGR046W                                     1.155988
Empty DataFrame
Columns: [ORF NAME, log2 ratio after a 5-h exposure to chitosan, orf]
Index: []
(1071, 1)
Original data dimensions: 4558 x 2
  ORF NAME 

In [8]:
original_data = pd.concat(original_data_list, axis=1)

In [11]:
original_data.index.name='orf'

In [12]:
original_data.head()

,data,data,data,data
orf,,,,
YAL002W,0.831695,NaN,0.908913,NaN
YAL004W,0.093487,NaN,-0.125043,NaN
YAL005C,0.361769,NaN,-0.170290,NaN
YAL007C,0.161059,NaN,0.068949,NaN
YAL008W,0.016272,NaN,0.067879,NaN


In [13]:
original_data.shape

(5616, 4)

# Prepare the final dataset

In [14]:
data = original_data.copy()

In [15]:
dataset_ids = [605, 5291, 5292, 5293]
datasets = datasets.reindex(index=dataset_ids)

In [16]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [17]:
data.head()

dataset_id,605,5291,5292,5293
data_type,value,value,value,value
orf,,,,
YAL002W,0.831695,NaN,0.908913,NaN
YAL004W,0.093487,NaN,-0.125043,NaN
YAL005C,0.361769,NaN,-0.170290,NaN
YAL007C,0.161059,NaN,0.068949,NaN
YAL008W,0.016272,NaN,0.067879,NaN


## Subset to the genes currently in SGD

In [18]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 24


In [19]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,605,5291,5292,5293
,data_type,value,value,value,value
gene_id,orf,,,,
2,YAL002W,0.831695,NaN,0.908913,NaN
1863,YAL004W,0.093487,NaN,-0.125043,NaN
4,YAL005C,0.361769,NaN,-0.170290,NaN
5,YAL007C,0.161059,NaN,0.068949,NaN
6,YAL008W,0.016272,NaN,0.067879,NaN


# Normalize

In [20]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [21]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [22]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,605,5291,5292,5293,605,5291,5292,5293
,data_type,value,value,value,value,valuez,valuez,valuez,valuez
gene_id,orf,,,,,,,,
2,YAL002W,0.831695,NaN,0.908913,NaN,1.509686,NaN,1.326076,NaN
1863,YAL004W,0.093487,NaN,-0.125043,NaN,0.121073,NaN,-0.222719,NaN
4,YAL005C,0.361769,NaN,-0.170290,NaN,0.625728,NaN,-0.290495,NaN
5,YAL007C,0.161059,NaN,0.068949,NaN,0.248181,NaN,0.067869,NaN
6,YAL008W,0.016272,NaN,0.067879,NaN,-0.024173,NaN,0.066265,NaN


# Print out

In [23]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [24]:
from IO.save_data_to_db3 import *

In [25]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/4 [00:00<?, ?it/s]

Deleting all datasets for PMID 17259547...
Inserting the new data...


100%|██████████| 4/4 [00:29<00:00,  7.29s/it]

Updating the data_modified_on field...
